응급의료시설 : 1
산후조리원 :2
병원 :3
산부인과 :4
보건소 :5 
정신건강센터 :6
개발자모드:7
단축키를 입력해주세요:  3


In [ ]:
#시각화 확인용
import pandas as pd
def config_thisdef(num1):
    if num1 ==1:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/emerback.csv', encoding = 'cp949')
        df_sample = df[['시군명','병원명/센터명','대표전화번호','위도','경도']]
        df_drop = df_sample.dropna(subset=['위도'])
        df_fn = df_drop.reset_index(drop=True)
        display(df_fn)
        
    elif num1 == 2:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/care_centerback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태구분코드','소재지시설전화번호','WGS84위도','WGS84경도']]
        df_drop_sample = df_sample.drop(index= df_sample[df_sample['영업상태구분코드']!=13].index) 
        df_ddrop = df_drop_sample.dropna(subset=['WGS84위도'])
        df_fn = df_ddrop.reset_index(drop=True)
        display(df_fn)
        
    elif num1 == 3:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/medicback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태명','소재지시설전화번호','위도','경도']] 
        df_drop_sample = df_sample[df_sample['영업상태명'].str.contains('영업')]
        df_ddrop = df_drop_sample.dropna(subset=['위도'])
        df_fn = df_ddrop.reset_index(drop=True)
        display(df_fn)
#진료내역중 산부인과가 있는 병원만 검색      
    elif num1 == 4:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/medicback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태명','소재지시설전화번호','위도','경도','진료과목내용정보']] 
        df_drop_sample = df_sample[df_sample['영업상태명'].str.contains('영업')]
        df_ddrop = df_drop_sample.dropna(subset=['위도'])
        df_seg = df_ddrop[df_ddrop['진료과목내용정보'].str.contains('산부인과')]
        df_fn = df_seg.reset_index(drop=True)
        display(df_fn)
        
    elif num1 == 5:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/public_medicback.csv', encoding = 'cp949')
        df_fn = df[['시군명','기관명','전화번호','WGS84위도','WGS84경도']]
        display(df_fn)
        
    elif num1 == 6:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/mentalback.csv', encoding = 'cp949')
        df_fn = df[['시군명','센터명','전화번호','WGS84위도','WGS84경도']]    
        display(df_fn)
num1 = int(input('응급의료시설 : 1\n산후조리원 :2\n병원 :3\n산부인과 :4\n보건소 :5 \n정신건강센터 :6\n단축키를 입력해주세요: '))
config_thisdef(num1)
        

In [16]:
#ver.1
#current_location()은 타 사이트애서 현재 사용자의 위치정보를 받아오는 함수이다.
#타 사이트에서 사용자의 현재 위치를 제대로 받아오지 못한경우 에러를 출력하고, 제대로 가져왔다면 lat와 lng를 실수타입으로 받아서 'lat':위도 'lng': 경도 라는 crd 딕셔너일로 받음.. 
# 위 데이터 파일들은 공통적으로 폐업여부 및 경위도 누락된 기관들을 코드 내 전처리 후 인덱스번호를 재정렬하고, 포리움으로 지도를 생성하는 코드임. 
#전처리 과정: 첫째로 인덱스를 필요한 정보만으로 가져와 샘플에 넣는다.
#영업상태코드나 영업상태명이 영업인 인덱스를 드랍에 넣는다. (폐업의 경우에만 사용)
#샘플에서 위도가 nan이라면 날린 후 드(드))랍에 넣는다.
#드(드)랍을 재정렬 한 후 fn에 넣는다. 

#코드 작동 과정
#for문을 통해 fn의 열 갯수에 해당하는 만큼 item을 돌림.
#fn의 [item]번째'위도,경도,이름,번호'를 각 변수로 생성한다. 
#경위도에 해당하는 변수는 그 위치에 마커를, 이름과 센터명은 인덱스를 생성해 지도상에서 마커를 클릭 시 정보사항이 나타난다. 
#코드가 돌아가면, 각 기관에 해당되는 단축키를 안내후 입력받는다 .그 후 번호에 해당하는 지도를 사용자 위치를 중심으로 생성한다. 
#사용자가 코드를 재 작동 하면 기존 맵센터값은 초기화 된다.
import folium
import pandas as pd
import requests, json 

def current_location():
    here_req = requests.get("http://www.geoplugin.net/json.gp")

    if (here_req.status_code != 200):
        print("현재좌표를 불러올 수 없음")
    else:
        location = json.loads(here_req.text)
        crd = {"lat": float(location["geoplugin_latitude"]), "lng": float(location["geoplugin_longitude"])}
    return crd
crd = current_location()
map_center = folium.Map(location= (crd['lat'], crd['lng']) , zoom_start=15)

     
def start_center(num):
    if num == 1:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/emerback.csv', encoding = 'cp949')
        df_sample = df[['시군명','병원명/센터명','대표전화번호','위도','경도']]
        df_drop = df_sample.dropna(subset=['위도'])
        df_fn = df_drop.reset_index(drop=True)
        
        for item in range(len(df_fn)):
            latitude = df_fn['위도'][item]
            longitude = df_fn['경도'][item]
            name = str(df_fn['병원명/센터명'][item])
            callNum = df_fn['대표전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center  
    elif num == 2:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/care_centerback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태구분코드','소재지시설전화번호','WGS84위도','WGS84경도']]
        df_drop_sample = df_sample.drop(index= df_sample[df_sample['영업상태구분코드']!=13].index) 
        df_ddrop = df_drop_sample.dropna(subset=['WGS84위도'])
        df_fn = df_ddrop.reset_index(drop=True)
        
        for item in range(len(df_fn)):
            latitude = df_fn['WGS84위도'][item]
            longitude = df_fn['WGS84경도'][item]
            name = str(df_fn['사업장명'][item])
            callNum = df_fn['소재지시설전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center        
    elif num == 3:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/medicback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태명','소재지시설전화번호','위도','경도']] 
        df_drop_sample = df_sample[df_sample['영업상태명'].str.contains('영업')]
        df_ddrop = df_drop_sample.dropna(subset=['위도'])
        df_fn = df_ddrop.reset_index(drop=True)
        
        for item in range(len(df_fn)):
            latitude = df_fn['위도'][item]
            longitude = df_fn['경도'][item]
            name = str(df_fn['사업장명'][item])
            callNum = df_fn['소재지시설전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center  
       
    elif num == 4:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/medicback.csv', encoding = 'cp949')
        df_sample = df[['시군명','사업장명','영업상태명','소재지시설전화번호','위도','경도','진료과목내용정보']] 
        df_drop_sample = df_sample[df_sample['영업상태명'].str.contains('영업')]
        df_ddrop = df_drop_sample.dropna(subset=['위도'])
        df_seg = df_ddrop[df_ddrop['진료과목내용정보'].str.contains('산부인과')]
        df_fn = df_seg.reset_index(drop=True)
        
        
        for item in range(len(df_fn)):
            latitude = df_fn['위도'][item]
            longitude = df_fn['경도'][item]
            name = str(df_fn['사업장명'][item])
            callNum = df_fn['소재지시설전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center  
    
    elif num == 5:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/public_medicback.csv', encoding = 'cp949')
        df_fn = df[['시군명','기관명','전화번호','WGS84위도','WGS84경도']] 
        for item in range(len(df_fn)):
            latitude = df_fn['WGS84위도'][item]
            longitude = df_fn['WGS84경도'][item]
            name = str(df_fn['기관명'][item])
            callNum = df_fn['전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center

    elif num == 6:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        df = pd.read_csv('C:/Users/김동혁/hyeok python/project file/mentalback.csv', encoding = 'cp949')
        df_fn = df[['시군명','센터명','전화번호','WGS84위도','WGS84경도']] 
        for item in range(len(df_fn)):
            latitude = df_fn['WGS84위도'][item]
            longitude = df_fn['WGS84경도'][item]
            name = str(df_fn['센터명'][item])
            callNum = df_fn['전화번호'][item]
            marker = folium.Marker([latitude, longitude], popup=(name,callNum) )
            marker.add_to(map_center)
        return map_center


num = int(input('응급의료시설 : 1\n산후조리원 :2\n병원 :3\n산부인과 :4\n보건소 :5 \n정신건강센터 :6\n단축키를 입력해주세요: '))
start_center(num)
            
        

응급의료시설 : 1
산후조리원 :2
병원 :3
산부인과 :4
보건소 :5 
정신건강센터 :6
단축키를 입력해주세요:  1
